<a href="https://colab.research.google.com/github/dakilaledesma/NCBG/blob/main/train/Multilabel_Parts.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
%%time
! unzip -q /content/drive/MyDrive/UNC/NCBG/sept1_classif_training.zip -d /content
! cp /content/drive/MyDrive/UNC/NCBG/qry_fsus_trainingdata_ai.xlsx /content/qry_fsus_trainingdata_ai.xlsx

CPU times: user 41.3 ms, sys: 20.7 ms, total: 62.1 ms
Wall time: 4.85 s


In [ ]:
! mv /content/content/images /content/images
! rm -rf /content/content

In [ ]:
! pip install -U tensorflow-gpu

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 578.0 MB 15 kB/s 
     |████████████████████████████████| 1.7 MB 56.1 MB/s 
     |████████████████████████████████| 5.9 MB 46.7 MB/s 
     |████████████████████████████████| 438 kB 66.4 MB/s 
  Attempting uninstall: tensorflow-estimator
    Found existing installation: tensorflow-estimator 2.8.0
    Uninstalling tensorflow-estimator-2.8.0:
      Successfully uninstalled tensorflow-estimator-2.8.0
  Attempting uninstall: tensorboard
    Found existing installation: tensorboard 2.8.0
    Uninstalling tensorboard-2.8.0:
      Successfully uninstalled tensorboard-2.8.0
  Attempting uninstall: keras
    Found existing installation: keras 2.8.0
    Uninstalling keras-2.8.0:
      Successfully uninstalled keras-2.8.0
  Attempting uninstall: gast
    Found existing installation: gast 0.5.3
    Uninstalling gast-0.5.3:
      Successfully uninstalled gast-0.5

In [ ]:
! apt install --allow-change-held-packages libcudnn8=8.1.0.77-1+cuda11.2

Reading package lists... Done
Building dependency tree       
Reading state information... Done
The following package was automatically installed and is no longer required:
  libnvidia-common-460
Use 'apt autoremove' to remove it.
The following packages will be REMOVED:
  libcudnn8-dev
The following held packages will be changed:
  libcudnn8
The following packages will be upgraded:
  libcudnn8
1 upgraded, 0 newly installed, 1 to remove and 18 not upgraded.
Need to get 430 MB of archives.
After this operation, 3,139 MB disk space will be freed.
Get:1 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  libcudnn8 8.1.0.77-1+cuda11.2 [430 MB]
Fetched 430 MB in 7s (59.6 MB/s)
(Reading database ... 155569 files and directories currently installed.)
Removing libcudnn8-dev (8.0.5.39-1+cuda11.1) ...
(Reading database ... 155547 files and directories currently installed.)
Preparing to unpack .../libcudnn8_8.1.0.77-1+cuda11.2_amd64.deb ...
Unpacking libcudnn8 (8.1.0.77-1+c

In [ ]:
! pip install tensorflow-addons

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 1.1 MB 2.6 MB/s 


In [ ]:
import pandas as pd

df = pd.read_excel("qry_fsus_trainingdata_ai.xlsx")

In [ ]:
print(df.columns)

Index(['file', 'directory', 'imagequality', 'image_context'], dtype='object')


In [ ]:
classes = list(set(sum([v.split(';') for v in df["image_context"] if type(v) == str], [])))
classes.sort()
class_dict = {v: i for i, v in enumerate(classes)}

In [ ]:
print(class_dict)

{'aspect': 0, 'bud': 1, 'fall-color': 2, 'flower': 3, 'fruit': 4, 'leaf': 5, 'leaf-underside': 6, 'other': 7, 'root': 8, 'seed': 9, 'side-by-side': 10, 'stem': 11, 'venation': 12, 'vestiture': 13, 'whole-plant': 14}


In [ ]:
import numpy as np

im_to_class = {}
for idx, row in df.iterrows():
  anno_bn = row["file"]
  try:
    classes = row["image_context"].split(';')
  except AttributeError:
    continue
  
  out_anno = np.zeros(15, dtype=int)
  for class_ in classes:
    class_idx = class_dict.get(class_)
    out_anno[class_idx] = 1
  
  im_to_class[anno_bn] = out_anno

In [ ]:
# from glob import glob
# import os
# from PIL import Image, UnidentifiedImageError
# from tqdm.notebook import tqdm

# X = []
# y = []
# for im_fn in glob("images/*.*"):
#   bn = os.path.basename(im_fn).split("__")[1].replace(".jpg", '')
#   bn = f"{bn}.jpg"
#   try:
#     y.append(im_to_class[bn])
#     im = Image.open(im_fn)
#     im = im.resize((331, 331))
#     im = np.array(im, dtype=int)
#   except KeyError or UnidentifiedImageError:
#     continue

#   X.append(im)

In [ ]:
from glob import glob
import os
from PIL import Image, UnidentifiedImageError
from tqdm.notebook import tqdm

flow_list = []
for im_fn in glob("images/*.*"):
  bn = os.path.basename(im_fn).split("__")[1].replace(".jpg", '')
  bn = f"{bn}.jpg"
  try:
    y_val = [str(v) for v in im_to_class[bn]]
  except KeyError:
    continue

  try:
    im = Image.open(im_fn)
  except UnidentifiedImageError:
    continue

  flow_list.append([im_fn] + y_val)

In [ ]:
flow_df = pd.DataFrame(np.array(flow_list), columns=["filenames"] + list(class_dict.keys()))

In [ ]:
flow_df = flow_df.astype({v: 'float' for v in class_dict.keys()})

In [ ]:
print(flow_df.head())

                        filenames  aspect  bud  fall-color  flower  fruit  \
0  images/5115__rtw_11319.jpg.jpg     0.0  0.0         0.0     1.0    0.0   
1   images/3180__eb_01388.jpg.jpg     0.0  0.0         0.0     1.0    0.0   
2   images/4603__kb_02686.jpg.jpg     0.0  0.0         0.0     0.0    0.0   
3  images/1196__bas_00826.jpg.jpg     1.0  0.0         0.0     0.0    0.0   
4  images/6070__rtw_21549.jpg.jpg     0.0  0.0         0.0     1.0    0.0   

   leaf  leaf-underside  other  root  seed  side-by-side  stem  venation  \
0   1.0             0.0    0.0   0.0   0.0           0.0   0.0       0.0   
1   1.0             0.0    0.0   0.0   0.0           0.0   0.0       0.0   
2   1.0             0.0    0.0   0.0   0.0           0.0   1.0       0.0   
3   0.0             0.0    0.0   0.0   0.0           0.0   0.0       0.0   
4   1.0             0.0    0.0   0.0   0.0           0.0   0.0       0.0   

   vestiture  whole-plant  
0        0.0          0.0  
1        0.0          0.

In [ ]:
print(flow_df.dtypes)

filenames          object
aspect            float64
bud               float64
fall-color        float64
flower            float64
fruit             float64
leaf              float64
leaf-underside    float64
other             float64
root              float64
seed              float64
side-by-side      float64
stem              float64
venation          float64
vestiture         float64
whole-plant       float64
dtype: object


In [ ]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.applications.convnext import ConvNeXtLarge
from tensorflow.keras.layers import Dense

ConvNeXt top layer configured as such

```
    if include_top:
        x = Head(num_classes=classes, name=model_name)(x)
        imagenet_utils.validate_activation(classifier_activation, weights)
    else:
        if pooling == "avg":
            x = layers.GlobalAveragePooling2D()(x)
        elif pooling == "max":
            x = layers.GlobalMaxPooling2D()(x)
        x = layers.LayerNormalization(epsilon=1e-6)(x)
```



In [ ]:
backbone = ConvNeXtLarge(input_shape=(448, 448, 3), include_top=False, weights='imagenet', pooling='avg')
model = Sequential()
model.add(backbone)
model.add(Dense(512, activation="relu"))
model.add(Dense(15, activation="sigmoid"))

In [ ]:
import random

def custom_prepro(npt):
  ra = random.randint(0, 3)
  npt = np.rot90(npt, ra)
  return npt

gen = tf.keras.preprocessing.image.ImageDataGenerator(
    horizontal_flip=True,
    vertical_flip=True,
    rotation_range=10,
    shear_range=10.0,
    zoom_range=0.2,
    preprocessing_function=custom_prepro
)

res = gen.flow_from_dataframe(
    dataframe=flow_df,
    x_col="filenames",
    y_col=list(class_dict.keys()),
    batch_size=8,
    seed=42,
    shuffle=True,
    class_mode="raw",
    target_size=(448,448))

Found 8489 validated image filenames.


In [ ]:
# i = 0
# for batch in gen.flow_from_dataframe(
#     dataframe=flow_df,
#     x_col="filenames",
#     y_col=list(class_dict.keys()),
#     seed=42,
#     shuffle=True,
#     class_mode="raw",
#     target_size=(448,448),
#     batch_size=1,
#     save_to_dir='preview', 
#     save_prefix='ex', 
#     save_format='jpeg'):
#       i += 1
#       if i > 20:
#           break  # otherwise the generator would loop indefinitely

In [ ]:
import tensorflow_addons as tfa

opt = tfa.optimizers.AdaBelief(learning_rate=1e-3)
model.compile(optimizer=opt, loss="binary_crossentropy", metrics="categorical_accuracy")
model.fit(res, epochs=80)

/usr/local/lib/python3.7/dist-packages/tensorflow_addons/utils/ensure_tf_install.py:67: UserWarning: Tensorflow Addons supports using Python ops for all Tensorflow versions above or equal to 2.7.0 and strictly below 2.10.0 (nightly versions are not supported). 
 The versions of TensorFlow you are currently using is 2.10.0 and is not supported. 
Some things might work, some things might not.
If you were to encounter a bug, do not file an issue.
If you want to make sure you're using a tested and supported configuration, either change the TensorFlow version or the TensorFlow Addons's version. 
You can find the compatibility matrix in TensorFlow Addon's readme:
https://github.com/tensorflow/addons
  UserWarning,


Epoch 1/80
1062/1062 [==============================] - 719s 594ms/step - loss: 0.1934 - categorical_accuracy: 0.5611
Epoch 2/80
1062/1062 [==============================] - 629s 592ms/step - loss: 0.1840 - categorical_accuracy: 0.5648
Epoch 3/80
1062/1062 [==============================] - 629s 592ms/step - loss: 0.1814 - categorical_accuracy: 0.5746
Epoch 4/80
1062/1062 [==============================] - 630s 593ms/step - loss: 0.1796 - categorical_accuracy: 0.5733
Epoch 5/80
1062/1062 [==============================] - 630s 593ms/step - loss: 0.1754 - categorical_accuracy: 0.5835
Epoch 6/80
1062/1062 [==============================] - 629s 592ms/step - loss: 0.1716 - categorical_accuracy: 0.5924
Epoch 7/80
1062/1062 [==============================] - 629s 592ms/step - loss: 0.1681 - categorical_accuracy: 0.5969
Epoch 8/80
1062/1062 [==============================] - 629s 592ms/step - loss: 0.1620 - categorical_accuracy: 0.6082
Epoch 9/80
1062/1062 [==============================] - 

In [ ]:
model.save("/content/drive/MyDrive/UNC/NCBG/mp_2b")